# [Day 15] Advanced Training : FastAPI Core Logic

**"견고한 백엔드는 입력값 검증과 에러 처리에서 시작됩니다."**

Advanced Part에서는 FastAPI 개발의 핵심인 **Pydantic 모델링**과 **의존성 주입(Dependency Injection)**, 그리고 **사용자 정의 예외 처리**를 연습합니다.

---

In [2]:
# 필요 라이브러리 설치 (코랩 등 환경에 따라 필요시 실행)
!pip install pydantic fastapi


[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from pydantic import BaseModel, field_validator, ValidationError
from typing import List, Optional
from datetime import datetime

# 실습을 위한 가상 DB 역할
fake_items_db = {
    1: {"name": "MacBook Pro", "price": 2500, "stock": 10},
    2: {"name": "iPad Air", "price": 800, "stock": 0}
}

### Q1. Pydantic 모델과 커스텀 유효성 검사

상품 등록 API에 사용할 `ItemCreate` Pydantic 모델을 정의하세요.

**요구사항:**
1. `name`: 문자열 (필수)
2. `price`: 정수 (필수)
3. `stock`: 정수 (필수, 기본값 0)
4. **유효성 검사 (`@field_validator`):**
    * `price`는 반드시 **0보다 커야** 합니다.
    * `stock`은 **음수가 될 수 없습니다**.
    * 조건 위반 시 적절한 `ValueError`를 발생시키세요.

* **Tip:** Pydantic V2에서는 `@field_validator` 데코레이터를 사용합니다.

In [5]:
# 여기에 코드를 작성하세요

class ItemCreate(BaseModel):
    # 필드 정의
    name: str
    price: int
    stock: int = 0

    # Validator 정의
    @field_validator("price")
    @classmethod
    def check_price(cls, value):
        if value <= 0:
            raise ValueError("가격은 0보다 커야 합니다.")
        return value
    @field_validator("stock")
    @classmethod
    def check_stock(cls, value):
        if value < 0:
            raise ValueError("개수는 음수일 수 없습니다.")
        return value
    


# 테스트 코드 (수정하지 마세요)
try:
    item = ItemCreate(name="Gaming Mouse", price=-100, stock=5)
except ValidationError as e:
    print("Validation Success! Error caught:", e)

Validation Success! Error caught: 1 validation error for ItemCreate
price
  Value error, 가격은 0보다 커야 합니다. [type=value_error, input_value=-100, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


### Q2. 의존성 주입 (Dependency Injection) 시뮬레이션

FastAPI의 핵심인 `Depends`를 흉내 내어 의존성을 주입하는 함수를 작성해 봅니다.

1. `get_db_session` 함수: "DB 연결 성공" 이라는 문자열을 반환하는 간단한 함수.
2. `create_item_service` 함수: 
    * `item` (Q1에서 만든 모델 인스턴스)과 `db` (DB 세션 문자열)를 인자로 받습니다.
    * "[DB 연결 성공] Item 'MacBook' saved!" 와 같은 문자열을 반환합니다.
3. **직접 호출:** `get_db_session()`의 결과를 변수에 담아 `create_item_service`에 넘겨주는 코드를 작성하세요.

* **의도:** FastAPI가 내부적으로 어떻게 `Depends`를 처리해서 넣어주는지 원리를 이해하는 과정입니다.

In [6]:
# 1. 의존성(Dependency) 함수: 필요한 '도구(DB 세션)'를 만들어주는 역할
def get_db_session() -> str:
    return "DB 연결 성공"

# 2. 메인 로직 함수: 도구(db)와 데이터(item)를 받아서 실제 작업을 처리하는 역할
def create_item_service(item: ItemCreate, db: str) -> str:
    # f-string을 사용하여 db 상태 문자열과 item 객체의 name 속성을 조합합니다.
    return f"[{db}] Item '{item.name}' saved!"

# 3. 직접 주입해보기 (실제로는 FastAPI가 뒤에서 자동으로 해주는 과정)
my_db = get_db_session()  # 1단계: 도구를 준비한다.
new_item = ItemCreate(name="MacBook", price=2500, stock=10) # 2단계: 데이터를 준비한다.

# 3단계: 준비된 도구와 데이터를 메인 함수에 '주입(Injection)' 한다.
result = create_item_service(item=new_item, db=my_db)

print(result)
# 출력 결과: [DB 연결 성공] Item 'MacBook' saved!

[DB 연결 성공] Item 'MacBook' saved!


### Q3. 예외 처리와 비즈니스 로직 (Try-Except)

상품을 구매하는 함수 `buy_item(item_id: int, quantity: int)`를 작성하세요.

**로직:**
1. `fake_items_db`에서 `item_id`로 상품을 찾습니다. (없으면 `KeyError` 발생 -> 잡아서 "상품 없음" 출력)
2. 재고(`stock`)가 주문 수량(`quantity`)보다 적으면 커스텀 예외 `OutOfStockError`를 발생시킵니다.
3. 성공하면 재고를 차감하고, 남은 재고량을 반환합니다.

**커스텀 예외 클래스:**
```python
class OutOfStockError(Exception):
    def __init__(self, message="재고 부족"):
        self.message = message
        super().__init__(self.message)
```

In [7]:
# 실습을 위한 가상 DB 역할 (참고용)
fake_items_db = {
    1: {"name": "MacBook Pro", "price": 2500, "stock": 10},
    2: {"name": "iPad Air", "price": 800, "stock": 0}
}

# 1. 커스텀 예외 클래스 정의
class OutOfStockError(Exception):
    def __init__(self, message="재고 부족"):
        self.message = message
        super().__init__(self.message)

# 2. 비즈니스 로직 함수
def buy_item(item_id: int, quantity: int):
    try:
        # Step 1: 상품 조회 (만약 없는 id를 넣으면 여기서 바로 KeyError가 터집니다)
        item = fake_items_db[item_id]
        
        # Step 2: 재고 검증 (비즈니스 로직)
        if item["stock"] < quantity:
            # 우리가 만든 '재고 부족' 에러를 강제로 발생(raise)시킵니다.
            raise OutOfStockError(f"재고가 부족합니다. (요청: {quantity}, 현재: {item['stock']})")
        
        # Step 3: 결제 성공 시 재고 차감
        item["stock"] -= quantity
        print(f"✅ 성공: {item['name']} {quantity}개 구매 완료. (남은 재고: {item['stock']})")
        
        return item["stock"]

    # Step 4: 에러 상황별 대처 (안전 그물)
    except KeyError:
        # DB에 없는 상품 번호를 찾을 때 나는 파이썬 기본 에러
        print("❌ 실패: 상품 없음 (존재하지 않는 상품 ID입니다.)")
        
    except OutOfStockError as e:
        # 우리가 if문에서 강제로 발생시킨 커스텀 에러
        print(f"❌ 실패: {e.message}")

# --- [테스트 코드] ---
print("--- 1. 정상 구매 테스트 ---")
buy_item(item_id=1, quantity=3)

print("\n--- 2. 재고 부족 테스트 ---")
buy_item(item_id=2, quantity=1)

print("\n--- 3. 없는 상품 테스트 ---")
buy_item(item_id=99, quantity=1)

--- 1. 정상 구매 테스트 ---
✅ 성공: MacBook Pro 3개 구매 완료. (남은 재고: 7)

--- 2. 재고 부족 테스트 ---
❌ 실패: 재고가 부족합니다. (요청: 1, 현재: 0)

--- 3. 없는 상품 테스트 ---
❌ 실패: 상품 없음 (존재하지 않는 상품 ID입니다.)
